In [1]:
# import the libraries:
import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf
from colorcet import fire, kbc, bmw, gray
import plotly.express as px
import plotly.io as pio
pio.renderers.default='browser'
from IPython.display import HTML
import chart_studio.plotly as py

In [1]:
# # Import the dataframe:
df = pd.read_csv("Path to the file...")
df

In [31]:
# Take access key:
token = ('pk.eyJ1Ijoic2hhZGlvdXMiLCJhIjoiY2w2Zjl6cTg1MWplbjNpcWx1MTZ6eHJqdyJ9.Q04PpZxldDlys-mlzKk4VA')
px.set_mapbox_access_token((token))

# Build an abstract canvas representing the space in which to plot data
cvs = ds.Canvas(plot_width=1000, plot_height=1000)

# project the longitude and latitude onto the canvas and
# map the data to pixels as points
aggs = cvs.points(df, x='x', y='y') 


# aggs is an xarray object
coords_lon, coords_lat = aggs.coords['x'].values, aggs.coords['y'].values


# Set the corners of the image that need to be passed to the mapbox
coordinates = [[coords_lon[0], coords_lat[0]],
               [coords_lon[-1], coords_lat[0]],
                [coords_lon[-1], coords_lat[-1]],
                [coords_lon[0], coords_lat[-1]]]



# Set the image:
img = tf.shade(aggs, how='eq_hist')[::-1].to_pil()

# add figure:
fig = px.scatter_mapbox(df, lat="y", lon="x", hover_name = "year",
                color_continuous_scale="greens",color= "NDVI",
                zoom = 12, center= dict(lat= 52.391842, lon= 13.063561),
                template='plotly_dark')


# add dropdown for the type of map:    
updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
        dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.75,
        xanchor = 'left',
        y = 0.05,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11),)])
        
    
# update the style of our map:
fig.update_layout(
    mapbox_style="white-bg",
        mapbox=dict(
        accesstoken=token,
        bearing=0,
        center=dict(
            lat=52.391842,
            lon=13.063561
        ),
        pitch=0,
        style='dark',
                                layers=[
                    {
                    "sourcetype": "image",
                    "source": img,
                    "coordinates": coordinates
                      }
                  ]),
    updatemenus=updatemenus)
    
fig.show()